In [ ]:
import matplotlib
matplotlib.use('Agg') # disable interactive matplotlib to save RAM

import glob
import os 
import shutil
import spikeinterface.core as sc
import spikeinterface.curation as scu
import spikeinterface.extractors as se
import spikeinterface.sorters as ss
import sys 

from tqdm.auto import tqdm

sys.path.append('src')
from utils import *

data_root = 'data'
nsx = 'ns4'
si_folder = f'{data_root}{os.sep}spikeinterface-0_98_2{os.sep}LongTerm-{nsx}'

In [ ]:
recording_paths = sorted(glob.glob(f'{si_folder}{os.sep}**{os.sep}raw', recursive=True))

for recording_path in (pbar := tqdm(recording_paths)):
    recording_stem = os.sep.join(recording_path.split(os.sep)[:-1])
    mouse, date, recording_name = recording_stem.split(os.sep)[-3:]
    pbar.set_description(recording_path)

    recording_si_path = f'{recording_stem}{os.sep}processed'
    processed_trace_plot_file = f'{recording_si_path}{os.sep}processed.png'
    sorting_si_path = f'{recording_stem}{os.sep}sorting-by-group'
    waveforms_si_path = f'{recording_stem}{os.sep}waveforms-by-group'
    units_si_path = f'{recording_stem}{os.sep}units-by-group'

    if not os.path.isfile(f'{recording_si_path}{os.sep}binary.json'):
        recording = sc.load_extractor(recording_path)    
        recording_processed = preprocess_recording(recording)

        shutil.rmtree(recording_si_path, ignore_errors=True)
        recording_processed.save(folder=recording_si_path, memory=memory_limit)

    recording_processed = sc.load_extractor(recording_si_path)
    if not os.path.isfile(processed_trace_plot_file):
        plot_traces(recording_processed.get_traces().T, recording_processed.sampling_frequency, blackrock_channel_indices, title=f'{mouse} -> {date} -> {recording_name}', savepath=processed_trace_plot_file, trace_gap=150)

    pbar.set_description(f'{recording_stem} preprocessed')

    if not os.path.isfile(f'{sorting_si_path}{os.sep}sorter_output{os.sep}firings.npz'):
        recording_processed = sc.load_extractor(recording_si_path)

        probegroup = create_probegroup(blackrock_channel_indices)
        recording_processed.set_probegroup(probegroup, group_mode='by_probe', in_place=True)

        sorting = ss.run_sorter_by_property(
            sorter_name='mountainsort4',
            recording=recording_processed,
            grouping_property='group',
            working_folder=sorting_si_path,
            mode_if_folder_exists='overwrite',
            **sorter_parameters,
        )
        os.makedirs(f'{sorting_si_path}{os.sep}sorter_output', exist_ok=True)
        se.NpzSortingExtractor.write_sorting(sorting, f'{sorting_si_path}{os.sep}sorter_output{os.sep}firings.npz')
    pbar.set_description(f'{recording_stem} sorted')

    if not os.path.isfile(f'{waveforms_si_path}{os.sep}templates_average.npy'):
        recording_processed = sc.load_extractor(recording_si_path)
        sorting = se.NpzSortingExtractor(f'{sorting_si_path}{os.sep}sorter_output{os.sep}firings.npz')
        # spikeinterface https://github.com/SpikeInterface/spikeinterface/pull/1378
        sorting = scu.remove_excess_spikes(sorting, recording_processed)
        sc.extract_waveforms(
            recording_processed, sorting, 
            folder=waveforms_si_path,
            ms_before=ms_before, ms_after=ms_after, max_spikes_per_unit=None,
            return_scaled=False,
            overwrite=True,
            use_relative_path=True,
        )
    pbar.set_description(f'{recording_stem} extracted')

    recording_processed, sorting, waveform_extractor, extremum_channels = read_sorted_results(recording_stem, read_sessions=False, by_group=True)

    os.makedirs(units_si_path, exist_ok=True)
    for unit_id in sorting.unit_ids:
        pbar.set_description(f'{recording_stem} Plotting [unit {unit_id} / {len(sorting.unit_ids)}]')
        unit_plot_file = f'{units_si_path}{os.sep}{unit_id}.png'
        if not os.path.isfile(unit_plot_file):
            plot_unit(waveform_extractor, extremum_channels, sorting, unit_id, blackrock_channel_indices, initdate=surgery_dates[mouse], savepath=unit_plot_file)
    break